In [1]:
import yaml
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import gc

In [2]:
def load_rating():
    gc.collect()
    with open(f'../data/books.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            yield data['reviewerID'], data['asin']

In [3]:
arr = []
i = 0
for data in tqdm(load_rating()):
    arr.append(data)

51311621it [09:35, 89114.52it/s] 


In [4]:
df = pd.DataFrame(arr, columns=['user','item'])

In [6]:
cnt_user = df.groupby('user').agg({'item' : 'count'})
cnt_book = df.groupby('item').agg({'user' : 'count'})

In [21]:
user_more_h = cnt_user[cnt_user['item'] > 100]
book_more_h = cnt_book[cnt_book['user'] > 100]

In [22]:
user_list = user_more_h.index
book_list = book_more_h.index

In [23]:
df_t = df.loc[df['user'].isin(user_list)]
df_t = df_t.loc[df['item'].isin(book_list)]

In [24]:
books = df_t['item'].unique()

In [25]:
def get_data(*keys):
    data = volume_info
    global err_toggle
    try :
        for key in keys:
            data = data[key]
        return data
    except :
        return None

In [26]:
with open('../data/google_api.json', 'a') as f:
    for i in tqdm(range(len(books))) :
        ISBN = books[i]
        if 'B' in ISBN :
            continue

        response = requests.get(f"https://www.googleapis.com/books/v1/volumes?q={ISBN}")


        try : volume_info = response.json()['items'][0]['volumeInfo']
        except :
            f.write(f'{{"isbn" : "{ISBN}", "authors" : "None", "publisher" : "None", "publish_date" : "None", "page_cnt" : "None", "categories" : "None", "average_rating" : "None", "rating_count" : "None", "synopsis" : "None"}}\n')
            continue   

        try :
            if volume_info['industryIdentifiers'][0]["identifier"] != ISBN:
                f.write(f'{{"isbn" : "{ISBN}", "authors" : "None", "publisher" : "None", "publish_date" : "None", "page_cnt" : "None", "categories" : "None", "average_rating" : "None", "rating_count" : "None", "synopsis" : "None"}}\n')
                continue
        except :
            f.write(f'{{"isbn" : "{ISBN}", "authors" : "None", "publisher" : "None", "publish_date" : "None", "page_cnt" : "None", "categories" : "None", "average_rating" : "None", "rating_count" : "None", "synopsis" : "None"}}\n')
            continue


        description = get_data('description')
        authors = get_data('authors')
        publisher = get_data("publisher")
        publish_date = get_data("publishedDate")
        page_cnt = get_data('pageCount')
        categories = get_data('categories')
        average_rating = get_data('averageRating')
        rating_count = get_data('ratingsCount')
        info_link = get_data('infoLink')
        synopsis = "None"
        
        if info_link != None : 
            html = BeautifulSoup(requests.get(info_link).text)
            try : 
                synopsis = html.select_one('#synopsistext').text
            except : synopsis = "None"

        
        f.write(f'{{"isbn" : "{ISBN}", "authors" : "{authors}", "publisher" : "{publisher}", "publish_date" : "{publish_date}", "page_cnt" : "{page_cnt}", "categories" : "{categories}", "average_rating" : "{average_rating}", "rating_count" : "{rating_count}", "synopsis" : "{synopsis}"}}\n')

 62%|██████▏   | 49335/79351 [16:43:24<10:10:29,  1.22s/it] 


KeyboardInterrupt: 

In [20]:
with open('../data/google_api.json', 'r') as f:
    for l in f.readlines():
        print(l)

{"isbn" : "0001384198", "authors" : "['Keith Moseley', 'Mabel Caroline Bragg']", "publisher" : "HarperCollins", "publish_date" : "1984", "page_cnt" : "8",         "categories" : "['Engines']", "average_rating" : "4", "rating_count" : "78", "synopsis" : ""}

{"isbn" : "0001381733", "authors" : "['Robert Louis Stevenson', 'Gyo Fujikawa']", "publisher" : "HarperCollins", "publish_date" : "1972", "page_cnt" : "110",         "categories" : "['English poetry']", "average_rating" : "4", "rating_count" : "41", "synopsis" : "Brian Wildsmith's stunning art is the backdrop to this newly revised selection of Robert Louis Stevenson's wonderful poetry for children. First published in 1885 and continuously in print since then, Stevenson's poetry captures the joy and whimsy of childhood and is deservedly a classic. This book is a garden of delightful, imaginative poetry, complete with fairies, animals, and plenty of fun. Make-believe that a bed is a ship and exchange pirate stories; swing from a favor